In [1]:
import pandas as pd
import re

import past_guest_module as pm

import sys
sys.path.append('../zoho_data_app/')

import main_module as m

%load_ext autoreload
%autoreload 2


In [2]:
path = r'C:\Users\fajar\Documents\Python\Data\check_country_code_analytic.xlsx'
df_analytic = pd.read_excel(path, dtype='string', sheet_name='Sheet1')
count_data_awal = df_analytic.shape[0]
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK057031,Karma Resorts,Other,Past Guests,Karma Kandara,PF276970,Mrs,Carrie,Matson,Female,...,2024-07-07 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KK046724,Karma Resorts,Other,Past Guests,Karma Kandara,PF256346,Mr,Brad John,Spicer,Male,...,2024-07-07 00:00:00,7,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
2,KK038095,Karma Resorts,Other,Past Guests,Karma Kandara,PF232691,Ms,Lee,Holyoake,Female,...,2024-07-12 00:00:00,14,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
3,KK055852,Karma Resorts,Other,Past Guests,Karma Kandara,PF273823,Mrs,Winny Wei - Hsin,Lee,Female,...,2024-07-08 00:00:00,8,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
4,KK054211,Karma Resorts,Other,Past Guests,Karma Kandara,PF270189,Ms,Henriette,Opperud,Female,...,2024-07-15 00:00:00,4,OTA,OTA,RETAIL,3,1,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,KSH017483,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF278177,Mrs,Emma,Prendergast,Female,...,2024-07-07 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
97,KK056866,Karma Resorts,Other,Past Guests,Karma Kandara,PF276600,Mr,William Haden,Eppard,Male,...,2024-07-07 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
98,KK056737,Karma Resorts,Other,Past Guests,Karma Kandara,PF276222,Mr,Burcu,Meci Bahceci,Male,...,2024-07-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
99,KB038168,Karma Resorts,Other,Past Guests,Karma Bavaria,PF253183,<NA>,Ida,Delin,Unknown,...,2024-07-09 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Phone Only,NO


In [3]:
df_analytic.columns

Index(['Booking_Id', 'Lead Brand', 'Lead Sub- Brand', 'Lead Source',
       'Resort_Name', 'ProfileRef', 'Salutation', 'First_Name', 'Surname',
       'Gender', 'Language', 'DOB', 'Street', 'Area', 'Town', 'County',
       'Country', 'Nationality', 'PostCode', 'TelNo', 'MobileNo', 'Email',
       'Exclude From Mailing', 'Opt In', 'Booking Status', 'DateArrival',
       'DateDepart', 'No. of Nights', 'Market_Segment', 'Media_Code',
       'Media Code Group', 'No. of Adult', 'No. of Children', 'No. of Infants',
       'Contact Type', 'Odyssey Members'],
      dtype='object')

In [4]:
df_analytic.fillna('empty', inplace=True)

In [5]:
path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Phone Members.xlsx'
df_phone = pd.read_excel(path, dtype='string', sheet_name='Sheet1')

path = r'G:\.shortcut-targets-by-id\0B_d3smBjzo04UUNBME5GOVhoMjA\_ODYSSEY\Email Members.xlsx'
df_email = pd.read_excel(path, dtype='string', sheet_name='Sheet1')


In [6]:
m.clean_space(df_email, 'Email')
df_email['Email'] = m.lowercase(df_email, 'Email')

In [7]:
# hitung jumlah duplicate
count_duplicate_general = df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])].shape[0]

df_analytic.loc[df_analytic.duplicated(subset=['TelNo','MobileNo','Email'])]

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
21,KB040980,Karma Resorts,Other,Past Guests,Karma Bavaria,PF265913,Herr,Manfred,Drescher,Male,...,2024-07-07 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
27,KB039949,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2024-07-07 00:00:00,2,COMP,V_OWNER,COMP,2,0,0,Email and Phone,NO
36,KB035770,Karma Resorts,Other,Past Guests,Karma Bavaria,PF243725,empty,Alexander,Schneider,Unknown,...,2024-07-14 00:00:00,5,OTA,OTA,RETAIL,3,1,0,Phone Only,NO
38,KB043089,Karma Resorts,Other,Past Guests,Karma Bavaria,PF232551,Herr,Mario,Althoff,Unknown,...,2024-07-10 00:00:00,3,DIR,WALKIN,RETAIL,1,0,0,Email and Phone,NO
42,KB040621,Karma Resorts,Other,Past Guests,Karma Bavaria,PF264273,empty,Lyaysan,Nurullina,Unknown,...,2024-07-13 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
45,KB040543,Karma Resorts,Other,Past Guests,Karma Bavaria,PF263927,empty,Wolfgang,Messner,Unknown,...,2024-07-14 00:00:00,2,OTA,OTA,RETAIL,1,1,0,Phone Only,NO
47,KB039180,Karma Resorts,Other,Past Guests,Karma Bavaria,PF258409,empty,Anna-Maria,Kessler,Unknown,...,2024-07-07 00:00:00,1,OTA,OTA,RETAIL,6,0,0,Phone Only,NO
70,KLM004275,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF261945,Mrs,Katie,Mcavoy,Female,...,2024-07-10 00:00:00,2,DIR,TEL,RETAIL,2,0,0,Email and Phone,NO
72,KSM010022,Karma Resorts,Other,Past Guests,Karma St Martins,PF067424,Mr,Karma St. Martin'S,Management,Male,...,2024-07-09 00:00:00,3,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
82,KLM001596,Karma Resorts,Other,Past Guests,Karma Lake of Menteith,PF278204,Miss,Michelle,Laing,Unknown,...,2024-07-08 00:00:00,1,COMP,WEDDING,GROUP,2,0,0,Email and Phone,NO


In [8]:
df_analytic.drop_duplicates(subset=['TelNo','MobileNo','Email'], inplace=True)
df_analytic.reset_index(drop=True, inplace=True)

# hitung jumlah setelah value duplicate dihapus
count_after_dedup_duplicate = df_analytic.shape[0]

df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK057031,Karma Resorts,Other,Past Guests,Karma Kandara,PF276970,Mrs,Carrie,Matson,Female,...,2024-07-07 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KK046724,Karma Resorts,Other,Past Guests,Karma Kandara,PF256346,Mr,Brad John,Spicer,Male,...,2024-07-07 00:00:00,7,HOUSE_USE,V_OWNER,COMP,4,0,0,Email and Phone,NO
2,KK038095,Karma Resorts,Other,Past Guests,Karma Kandara,PF232691,Ms,Lee,Holyoake,Female,...,2024-07-12 00:00:00,14,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
3,KK055852,Karma Resorts,Other,Past Guests,Karma Kandara,PF273823,Mrs,Winny Wei - Hsin,Lee,Female,...,2024-07-08 00:00:00,8,HOUSE_USE,V_OWNER_NRB,COMP,4,0,0,Email and Phone,NO
4,KK054211,Karma Resorts,Other,Past Guests,Karma Kandara,PF270189,Ms,Henriette,Opperud,Female,...,2024-07-15 00:00:00,4,OTA,OTA,RETAIL,3,1,0,Phone Only,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,KSH017483,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF278177,Mrs,Emma,Prendergast,Female,...,2024-07-07 00:00:00,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
87,KK056866,Karma Resorts,Other,Past Guests,Karma Kandara,PF276600,Mr,William Haden,Eppard,Male,...,2024-07-07 00:00:00,1,HOUSE_USE,V_OWNER_NRB,COMP,2,0,0,Email and Phone,NO
88,KK056737,Karma Resorts,Other,Past Guests,Karma Kandara,PF276222,Mr,Burcu,Meci Bahceci,Male,...,2024-07-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
89,KB038168,Karma Resorts,Other,Past Guests,Karma Bavaria,PF253183,empty,Ida,Delin,Unknown,...,2024-07-09 00:00:00,1,OTA,OTA,RETAIL,2,2,0,Phone Only,NO


In [9]:
find_member_booking = df_analytic.copy()

find_member_booking.loc[find_member_booking['TelNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile' 
find_member_booking.loc[find_member_booking['MobileNo'].isin(df_phone['Mobile']), 'Same Value Find'] = 'Member-Mobile'

find_member_booking.loc[find_member_booking['Email'].isin(df_email['Email']), 'Same Value Find'] = 'Member-Email'
find_member_booking.drop(find_member_booking.loc[find_member_booking['Same Value Find'].isna()].index, inplace=True)

find_member_booking.replace('empty', '', inplace=True)

In [10]:
find_member_booking.to_csv(r'C:\Users\fajar\Documents\Python\Data\cek_member_zoho_analytic.csv', index=False)

In [11]:
count_dedup_member = df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])].shape[0]


df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])]


,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
25,KB037478,Karma Resorts,Other,Past Guests,Karma Bavaria,PF147772,Frau,Claudia,Goebl,Female,...,2024-07-07 00:00:00,1,COMP,GER_OWE,COMP,1,0,0,Email and Phone,NO
41,KB041040,Karma Resorts,Other,Past Guests,Karma Bavaria,PF171431,Frau,Olga,Hottmann,Female,...,2024-07-07 00:00:00,3,COMP,V_OWNER,COMP,2,0,0,Email and Phone,NO
42,KB041039,Karma Resorts,Other,Past Guests,Karma Bavaria,PF216954,Herr,David,Raguin,Male,...,2024-07-07 00:00:00,2,COMP,GER_OWE,COMP,2,1,1,Email and Phone,NO
46,KB038009,Karma Resorts,Other,Past Guests,Karma Bavaria,PF228618,Ms,Caroline,Impler,Female,...,2024-07-07 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
48,KB043033,Karma Resorts,Other,Past Guests,Karma Bavaria,PF018070,empty,Rolf,Badenhoop,Male,...,2024-07-13 00:00:00,2,COMP,GER_OWE,COMP,2,0,0,Email Only,NO
55,KK056282,Karma Resorts,Other,Past Guests,Karma Kandara,PF275056,Mr,Weshley,Jehian,Male,...,2024-07-08 00:00:00,3,DIGITAL,WEBSITE,DIGITAL,6,0,0,Email and Phone,NO
63,KSM011334,Karma Resorts,Other,Past Guests,Karma St Martins,PF067424,Mr,Karma St. Martin'S,Management,Male,...,2024-07-14 00:00:00,13,COMP,STAFF,COMP,2,0,0,Email and Phone,NO
82,KSM013274,Karma Resorts,Other,Past Guests,Karma St Martins,PF241911,Mr,Chris,Pervey,Unknown,...,2024-07-07 00:00:00,1,DIR,WALKIN,RETAIL,2,0,0,Email and Phone,NO


In [12]:
df_analytic.drop(df_analytic.loc[
    
    df_analytic['TelNo'].isin(df_phone['Mobile']) | 
    df_analytic['MobileNo'].isin(df_phone['Mobile']) |

    df_analytic['Email'].isin(df_email['Email'])
].index, inplace=True)

df_analytic.reset_index(drop=True, inplace=True)

count_after_dedup_member = df_analytic.shape[0]

In [13]:
count_data_akhir = df_analytic.shape[0]

In [14]:
df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK057031,Karma Resorts,Other,Past Guests,Karma Kandara,PF276970,Mrs,Carrie,Matson,Female,...,2024-07-07 00:00:00,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KB042681,Karma Resorts,Other,Past Guests,Karma Bavaria,PF275514,Frau,Monika,Gawlok,Female,...,2024-07-07 00:00:00,1,OTA,OTA,RETAIL,2,0,1,Email and Phone,NO
2,KB041219,Karma Resorts,Other,Past Guests,Karma Bavaria,PF267164,Mr,Jacek,Kuhn,Male,...,2024-07-07 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
3,KB039677,Karma Resorts,Other,Past Guests,Karma Bavaria,PF232551,Herr,Mario,Althoff,Unknown,...,2024-07-07 00:00:00,7,DIR,EMAIL_MRKTNG,DIGITAL,2,0,0,Email and Phone,NO
4,KB036903,Karma Resorts,Other,Past Guests,Karma Bavaria,PF248659,Frau,Astrid,Borner,Female,...,2024-07-07 00:00:00,1,OTA,OTA,RETAIL,4,3,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,KK057203,Karma Resorts,Other,Past Guests,Karma Kandara,PF277560,Mr,Peter,Cao,Male,...,2024-07-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
79,KK054211,Karma Resorts,Other,Past Guests,Karma Kandara,PF270189,Ms,Henriette,Opperud,Female,...,2024-07-15 00:00:00,4,OTA,OTA,RETAIL,3,1,0,Phone Only,NO
80,KK056737,Karma Resorts,Other,Past Guests,Karma Kandara,PF276222,Mr,Burcu,Meci Bahceci,Male,...,2024-07-15 00:00:00,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
81,KSM009398,Karma Resorts,Other,Past Guests,Karma St Martins,PF067354,empty,Mary,Norman,Female,...,2024-07-15 00:00:00,2,COMP,TEL,RETAIL,2,0,0,Email and Phone,NO


In [15]:
df_analytic['DOB'].loc[df_analytic['DOB']=='']

Series([], Name: DOB, dtype: string)

In [16]:
df_analytic['DOB'].replace('empty', '', inplace=True)
dob_con_date = pd.to_datetime(df_analytic['DOB'], errors='coerce')
dob_date = dob_con_date.dt.strftime('%Y-%m-%d')

arr_con_date = pd.to_datetime(df_analytic['DateArrival'])
arr_date = arr_con_date.dt.strftime('%Y-%m-%d')

dep_con_date = pd.to_datetime(df_analytic['DateDepart'])
dep_date = dep_con_date.dt.strftime('%Y-%m-%d')

df_analytic['DOB'] = dob_date
df_analytic['DateArrival'] = arr_date
df_analytic['DateDepart'] = dep_date


In [17]:
df_analytic.replace('empty', '', inplace=True)

df_analytic.sort_values('DateDepart', ascending=True, inplace=True)
df_analytic.reset_index(drop=True, inplace=True)
df_analytic

,Booking_Id,Lead Brand,Lead Sub- Brand,Lead Source,Resort_Name,ProfileRef,Salutation,First_Name,Surname,Gender,...,DateDepart,No. of Nights,Market_Segment,Media_Code,Media Code Group,No. of Adult,No. of Children,No. of Infants,Contact Type,Odyssey Members
0,KK057031,Karma Resorts,Other,Past Guests,Karma Kandara,PF276970,Mrs,Carrie,Matson,Female,...,2024-07-07,2,HOUSE_USE,V_OWNER_NRB,COMP,3,0,0,Email and Phone,NO
1,KSH017483,Karma Resorts,Other,Past Guests,Karma Salford Hall,PF278177,Mrs,Emma,Prendergast,Female,...,2024-07-07,1,DIR,WEDDING,GROUP,2,0,0,Email Only,NO
2,KB039869,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278248,Herr,Daniel,Hecht,Male,...,2024-07-07,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
3,KB039869,Karma Resorts,Other,Past Guests,Karma Bavaria,PF261399,Herr,Marc,Prester,Male,...,2024-07-07,1,OTA,OTA,RETAIL,4,0,0,Email and Phone,NO
4,KB040725,Karma Resorts,Other,Past Guests,Karma Bavaria,PF278220,Frau,Fuer,Kälber,Female,...,2024-07-07,1,OTA,OTA,RETAIL,3,0,0,Email and Phone,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,KK057149,Karma Resorts,Other,Past Guests,Karma Kandara,PF277351,Mrs,Qnm,Yjh,Female,...,2024-07-15,1,DIGITAL,WEBSITE,DIGITAL,2,0,0,Email and Phone,NO
79,KK057203,Karma Resorts,Other,Past Guests,Karma Kandara,PF277560,Mr,Peter,Cao,Male,...,2024-07-15,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO
80,KK054211,Karma Resorts,Other,Past Guests,Karma Kandara,PF270189,Ms,Henriette,Opperud,Female,...,2024-07-15,4,OTA,OTA,RETAIL,3,1,0,Phone Only,NO
81,KK056737,Karma Resorts,Other,Past Guests,Karma Kandara,PF276222,Mr,Burcu,Meci Bahceci,Male,...,2024-07-15,1,OTA,OTA,RETAIL,2,0,0,Phone Only,NO


In [18]:
print("==========================================================")
print("Summary")
print("==========================================================")
print("Jumlah Data Setelah Script Past Guest Sebelumnya : "+str(count_data_awal))
print("Jumlah Data Yang Duplicate : "+str(count_duplicate_general))
print("Jumlah Data Setelah Hapus Duplicate : "+str(count_after_dedup_duplicate))
print("Jumlah Data Yang Termasuk Member: "+str(count_dedup_member))
print("Jumlah Data Setelah Hapus Member : "+str(count_after_dedup_member))
print("Jumlah Data Akhir : "+str(count_data_akhir))

Summary
Jumlah Data Setelah Script Past Guest Sebelumnya : 101
Jumlah Data Yang Duplicate : 10
Jumlah Data Setelah Hapus Duplicate : 91
Jumlah Data Yang Termasuk Member: 8
Jumlah Data Setelah Hapus Member : 83
Jumlah Data Akhir : 83


In [19]:
df_analytic.to_excel(r'C:\Users\fajar\Documents\Python\Data\final_analytics.xlsx', index=False)